In [1]:
%load_ext autoreload
%autoreload 2

# import torch to create dataloader used later in project

import torch
import cv2

import matplotlib.pyplot as plt
%matplotlib inline 

# autoloads dependancies when edits are made


In [2]:
# HYPERPARAMETERS

LEARNING_RATE = 1e-4
BATCH_SIZE = 1
NUM_EPOCHS = 1

IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
CHANNEL_NUM = 3

IMAGE_DIMS = (IMAGE_HEIGHT, IMAGE_WIDTH, CHANNEL_NUM)


In [3]:
# HARDWARE SETUP

# Checks CUDA to see if it is operational

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

NUM_WORKERS = 0

PIN_MEMORY = True

print(DEVICE)

cuda


In [4]:
# Directory

# reading the number of images in each domain

import os, os.path

IMG_DIR = os.getcwd() +"\\images"

LANDSCAPE_DIR = IMG_DIR + "\\landscape"

ABSTRACT_DIR = IMG_DIR + "\\monet_jpg"

from PIL import Image

for name in os.listdir(LANDSCAPE_DIR):
    directory = LANDSCAPE_DIR + "\\" + name
    image = Image.open(directory)
    if image.mode != "RGB":
        print("Incompatible data type!")
        
def get_number_of_images(DIR):
    return len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])

LANDSCAPE_IMAGE_COUNT = get_number_of_images(LANDSCAPE_DIR)
ABSTRACT_IMAGE_COUNT = get_number_of_images(ABSTRACT_DIR)

print(f"There are {LANDSCAPE_IMAGE_COUNT} images in the landscape directory")
print(f"There are {ABSTRACT_IMAGE_COUNT} images in the abstract directory")


There are 4315 images in the landscape directory
There are 300 images in the abstract directory


In [5]:
# now we instantiate the loaders that will prepare our images for training

from dataloader import *
from torch.utils.data import DataLoader


def get_loaders(
        landscape_dir,
        abstract_dir,
        batch_size,
        landscape_transform,
        abstract_transform,
        num_workers,
        pin_memory,
        landscape_shuffle,
        abstract_shuffle
    ):
    
    landscape_ds = ImageDataset(image_dir = landscape_dir, transform=landscape_transform
    )
    
    landscape_loader = DataLoader(
        landscape_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=landscape_shuffle
    )
    
    abstract_ds = ImageDataset(image_dir=abstract_dir, transform=abstract_transform
    )
    
    abstract_loader = DataLoader(
        abstract_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=abstract_shuffle
    )
    
    
    return landscape_loader, abstract_loader

In [6]:
# CREATE TRAINING_VALIDATION_TESTING SETS

# TRAINING SET IS 70%
# VALIDATION SET IS 20%
# TESTING SET IS 10%
import albumentations as A
from albumentations.pytorch import ToTensorV2

train_size = int(LANDSCAPE_IMAGE_COUNT * 0.7)
validation_size=  int(LANDSCAPE_IMAGE_COUNT * 0.2)
testing_size = LANDSCAPE_IMAGE_COUNT - train_size - validation_size

assert LANDSCAPE_IMAGE_COUNT == (train_size + validation_size + testing_size), "unequal training_validation_test partitions"

landscape_transform = A.Compose(
            [
                A.HorizontalFlip(p=0.5),
                A.ShiftScaleRotate(scale_limit=0.20, rotate_limit=10, shift_limit=0.1, p=0.5, border_mode=cv2.BORDER_REFLECT),
                A.GridDistortion(p=0.5),
                A.Normalize(
                    mean=[0],
                    std=[1],
                    
                ),
                A.Resize(IMAGE_HEIGHT, IMAGE_WIDTH),
                ToTensorV2(),

            ]
        )

abstract_transform = A.Compose(
            [
                A.HorizontalFlip(p=0.5),
                A.ShiftScaleRotate(scale_limit=0.20, rotate_limit=10, shift_limit=0.1, p=0.5, border_mode=cv2.BORDER_REFLECT),
                A.GridDistortion(p=0.5),
                A.Normalize(
                    mean=[0],
                    std=[1],
                    
                ),
                A.Resize(IMAGE_HEIGHT, IMAGE_WIDTH),
                ToTensorV2(),

            ]
        )

landscape_loader, abstract_loader = get_loaders(
                    LANDSCAPE_DIR,
                    ABSTRACT_DIR,
                    BATCH_SIZE,
                    landscape_transform,
                    abstract_transform,
                    NUM_WORKERS,
                    PIN_MEMORY,
                    True,
                    True
                        
                )

# ls_training_loader, validation_loader, testing_loader = torch.utils.data.random_split(landscape_loader.dataset, [train_size, validation_size, testing_size])
# train_dataset, val_dataset = get_loaders()



In [7]:
# for x in ls_training_loader:
#     pass
# #     plt.imshow(data, interpolation='nearest')
#     plt.show()

In [8]:
import random
from PIL import Image
# from matplotlib import cm
from torchvision.transforms import ToPILImage

def demo_image_from_loader(loader):
    for idx, x in enumerate(loader):
        x = x.squeeze(0)
        print(x.shape)
        img = ToPILImage()(x)
        img.show()
        break
        
def demo_image(tensor):
    img = ToPILImage()(tensor)
    img.show()
    
    
# demo_image_from_loader(landscape_loader)
# demo_image_from_loader(abstract_loader)

In [9]:
from keras.initializers import RandomNormal
from keras.models import Input

from utils import generate_real_samples, generate_fake_samples, update_image_pool
# from models import Discriminator

# we create our discriminator and generator networks next,


from models import *

from dataloader import *

# d_model_ls = define_discriminator(IMAGE_DIMS, 64)
# d_model_art = define_discriminator(IMAGE_DIMS, 64)

# g_model_ls = define_generator(IMAGE_DIMS, blocks=6, f=64)
# g_model_art = define_generator(IMAGE_DIMS, blocks=6, f=64)

# img_l, targets_l = generate_real_samples(landscape_loader, 5)
# img_a, targets_a = generate_real_samples(abstract_loader, 5)

# print(img_l.shape)
# print(img_a.shape)

# img_l_f, targets_l_f = generate_fake_samples(g_model_ls, img_l)

# preds_l = g_model_ls(img_l)
# preds_a = g_model_art(img_a)



# print(preds_l.shape)
# print(preds_l.shape)



C:\ProgramData\Anaconda3\envs\ai-art-translation\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.6.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [10]:
# 

    

# print(landscape_loader[1].shape)
# pred=model(landscape_loader[1].unsqueeze(0))

# print(torch.argmax(pred))
# print(pred)
# print(pred.shape)

In [11]:

# img, targets = generate_real_samples(landscape_loader, 5)

# print(img.shape)
# print(targets)

# fake_img, fake_targets = generate_fake_samples(g_model_ls, img)



# print(img.shape)
# print(targets)

# print(fake_img.shape)
# print(fake_targets)

# X_realLs, y_realLs = generate_real_samples(landscape_loader, 5)
# X_realArt, y_realArt = generate_real_samples(abstract_loader, 5)
        
        
# pool = update_image_pool([], fake_img)


In [12]:
# def train_fn(loader, model, optimizer, loss_fn, scaler)

In [15]:
# The ideal size is Batch, Height, Width, Channels

from tensorflow.keras.optimizers import Adam

def save_models(step, g_model_LstoArt, g_model_ArttoLs):   
    filename1 = 'g_model_LstoArt_%06d.h5' % (step+1)
    
    filename2 = 'g_model_ArttoLs_%06d.h5' % (step+2)
    
    g_model_LstoArt.save(filename1)
    g_model_ArttoLs.save(filename2)
    
    print('>Saved: %s and %s' % (filename1, filename2))

def define_composite_model(g_model_1, d_model, g_model_2, image_shape):
    
    g_model_1.trainable = True
    
    d_model.trainable = False
    
    g_model_2.trainable = False
    
    # two composite models relate to steps to improve each generator
    
    ## TODO ##
    
    # get image from dataloader, I can define if it is real or not later
    #

    input_gen = Input(shape=image_shape)
    gen1_out = g_model_1(input_gen)
    output_d = d_model(gen1_out)
    
    # here we take an authentic image and assess if the output of a generator is real or fake
    ##########
    
    # identity element
    input_id = Input(shape=image_shape)
    output_id = g_model_1(input_id)
    
    # identity function should get nothing occur to it, as it is Going from A -> A
    
    # forward cycle
    output_f = g_model_2(gen1_out)
    
    # this can be compared to input_img to determine if details of object is retained.
    
    # backward cycle
    gen2_out = g_model_2(input_id)
    output_b = g_model_1(gen2_out)
    
    model = Model([input_gen, input_id], [output_d, output_id, output_f, output_b])
    
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    
    model.compile(loss=['mse', 'mae', 'mae', 'mae'], loss_weights=[1,5,10,10], optimizer=opt)
    
    return model

def train(d_model_Ls, d_model_Art, g_model_LstoArt, g_model_ArttoLs, c_model_LstoArt, c_model_ArttoLs, landscape_loader, abstract_loader):
    n_epochs, n_batch = NUM_EPOCHS, BATCH_SIZE
    
    
#     poolLs, poolArt = list(), list()
    
    bat_per_epo = int(LANDSCAPE_IMAGE_COUNT/ n_batch)
    
    n_steps = bat_per_epo * n_epochs
    
    for i in range(n_steps):
        # selecting batches of real samples
        X_realLs, y_realLs = generate_real_samples(landscape_loader, n_batch)
        X_realArt, y_realArt = generate_real_samples(abstract_loader, n_batch)
        
        # generate a batch of fake samples
        X_fakeLs, y_fakeLs = generate_fake_samples(g_model_ArttoLs, X_realArt)
        X_fakeArt, y_fakeArt = generate_fake_samples(g_model_LstoArt, X_realLs)
        # update fakes from pool
        
#         X_fakeLs = update_image_pool(poolLs, X_fakeLs)
#         X_fakeArt = update_image_pool(PoolArt, X_fakeArt)
        
        g_loss2, _, _, _, _ = c_model_ArttoLs.train_on_batch([X_realArt, X_realLs], [y_realLs,
                                                                                    X_realLs,
                                                                                    X_realArt,
                                                                                    X_realLs])
        
        dLs_loss1 = d_model_Ls.train_on_batch(X_realLs, y_realLs)
        dLs_loss2 = d_model_Ls.train_on_batch(X_fakeLs, y_fakeLs)
        
        g_loss1, _, _, _, _ = c_model_LstoArt.train_on_batch([X_realLs, X_realArt], [y_realArt,
                                                                                    X_realArt,
                                                                                    X_realLs,
                                                                                    X_realArt])
        dArt_loss1 = d_model_Art.train_on_batch(X_realArt, y_realArt)
        dArt_loss2 = d_model_Art.train_on_batch(X_fakeArt, y_fakeArt)
        
        if i % (100) == 0:
            print("Step {} of {} completed!".format((i+1), n_steps))
            save_models(i+1, g_model_LstoArt, g_model_ArttoLs)


    
image_shape = (IMAGE_HEIGHT, IMAGE_WIDTH, 3)

g_model_ls_to_art = define_generator(image_shape, blocks = 6, f=64)
g_model_art_to_ls = define_generator(image_shape, blocks = 6, f=64)
# g_model_ls_to_art.summary()

d_model_ls = define_discriminator(image_shape, filters=64)
d_model_art = define_discriminator(image_shape, filters=64)

c_model_ls_to_art = define_composite_model(g_model_ls_to_art, d_model_art, g_model_art_to_ls, image_shape)
c_model_art_to_ls = define_composite_model(g_model_art_to_ls, d_model_ls, g_model_ls_to_art, image_shape)




train(d_model_ls, d_model_art, g_model_ls_to_art
      , g_model_art_to_ls, c_model_ls_to_art
      , c_model_art_to_ls, landscape_loader, abstract_loader)


    

Step 1 of 4315 completed!
>Saved: g_model_LstoArt_000002.h5 and g_model_ArttoLs_000003.h5
Step 101 of 4315 completed!
>Saved: g_model_LstoArt_000102.h5 and g_model_ArttoLs_000103.h5


KeyboardInterrupt: 

In [ ]:

    
def summarize_performance(step, g_model, trainX, name, n_samples=5):
    X_in, _ = generate_real_samples(trainX, n_samples)
    
    X_out, _ = generate_fake_samples(g_model, X_in)
    

In [ ]:
# img, targets = generate_real_samples(landscape_loader, 5)

fake_img, fake_targets = generate_fake_samples(g_model_ls, img)

print(img.shape)
print(targets)

print(fake_img.shape)
print(fake_targets)

pool = update_image_pool([], fake_img)


In [ ]:
# next steps are to creates batches of real sample functions, from domain ls and art, and their respective labels (Smooth labelling?)

# then i do the same but for fake data (structural change)




